In [3]:
# !pip install detecto

In [182]:
import glob
import numpy as np
import io

import skimage
from PIL import Image
from detecto.core import Model
from detecto.visualize import detect_live, detect_video, plot_prediction_grid
from detecto.core import DataLoader, Dataset
from detecto.utils import read_image, xml_to_csv, normalize_transform

import torchvision.transforms as T
# from torchvision.models.detection import 

import matplotlib.pyplot as plt

In [3]:
model = Model()

In [175]:
# !du -sh /Users/haridas/.cache/torch/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth

In [57]:
# detect_live(model)
train_dir = "/Users/haridas/Projects/pic2card/data/train_and_test/train/"
test_dir = "/Users/haridas/Projects/pic2card/data/train_and_test/test"

In [31]:
# plot_prediction_grid
train_labels = xml_to_csv(
    '/Users/haridas/Projects/pic2card/data/train_and_test/train/',
    '/Users/haridas/Projects/pic2card/data/train_and_test/train_label.csv'
)
val_labels = xml_to_csv(
    '/Users/haridas/Projects/pic2card/data/train_and_test/test/',
    '/Users/haridas/Projects/pic2card/data/train_and_test/test_label.csv'
)

In [210]:
# Image reader and pre-processing pipeline.
transformer = T.Compose([
    T.ToPILImage(),
    lambda image: image.convert("RGB"),
    T.ToTensor(),
    normalize_transform()
])

In [211]:
dataset = Dataset(
    '/Users/haridas/Projects/pic2card/data/train_and_test/train_label.csv',
    image_folder='/Users/haridas/Projects/pic2card/data/train_and_test/train/',
    transform=transformer
)


val_dataset = Dataset(
    '/Users/haridas/Projects/pic2card/data/train_and_test/test_label.csv',
    image_folder='/Users/haridas/Projects/pic2card/data/train_and_test/test/',
    transform=transformer
)

In [193]:
# Dataset?
# for images, labels in dataset:
#     break
img = read_image(
    "/Users/haridas/Projects/pic2card/data/train_and_test/train/3.png"
)
# transformer = T.Compose([T.ToTensor(), normalize_transform()])
# np.unique([read_image(i).shape[2] for i in glob.glob(f"{train_dir}/*.png")])
# T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])(img)

In [216]:
# new_model = Model(labels['class'].unique().tolist())

In [157]:
# dataset._csv.describe()

In [209]:
# new_model.fit(dataset)